In [1]:
# Familiar imports
import numpy as np
import pandas as pd
import random
import os
import time
from pathlib import Path

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder

import lightgbm as lgb
#import xgboost as xgb
#import catboost as ctb

import warnings
warnings.simplefilter('ignore')

# Parameters

In [2]:
target = 'target'

DEBUG = False

if DEBUG:
    N_ESTIMATORS = 1
    N_SPLITS = 2
    SEED = 17
    CVSEED = 17
    EARLY_STOPPING_ROUNDS = 1
    VERBOSE = 100
    #N_ITERS = 2
else:
    N_SPLITS = 10
    N_ESTIMATORS = 20000
    EARLY_STOPPING_ROUNDS = 200
    VERBOSE = 1000
    SEED = 17
    CVSEED = 17
    #N_ITERS = 10

In [3]:
def set_seed(seed=17):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    
set_seed(SEED)

# Load data

In [4]:
# Load the training data
train = pd.read_csv("../input/30-days-of-ml/train.csv")
test = pd.read_csv("../input/30-days-of-ml/test.csv")

# Preview the data
train.head()

,id,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,...,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,target
0,1,B,B,B,C,B,B,A,E,C,...,0.400361,0.160266,0.310921,0.389470,0.267559,0.237281,0.377873,0.322401,0.869850,8.113634
1,2,B,B,A,A,B,D,A,F,A,...,0.533087,0.558922,0.516294,0.594928,0.341439,0.906013,0.921701,0.261975,0.465083,8.481233
2,3,A,A,A,C,B,D,A,D,A,...,0.650609,0.375348,0.902567,0.555205,0.843531,0.748809,0.620126,0.541474,0.763846,8.364351
3,4,B,B,A,C,B,D,A,E,C,...,0.668980,0.239061,0.732948,0.679618,0.574844,0.346010,0.714610,0.540150,0.280682,8.049253
4,6,A,A,A,C,B,D,A,E,A,...,0.686964,0.420667,0.648182,0.684501,0.956692,1.000773,0.776742,0.625849,0.250823,7.972260


In [5]:
# Low MI scores
for pp in ["cat2","cat3","cat4","cat6"]:
    train.pop(pp)
    test.pop(pp)
train.head()

,id,cat0,cat1,cat5,cat7,cat8,cat9,cont0,cont1,cont2,...,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,target
0,1,B,B,B,E,C,N,0.201470,-0.014822,0.669699,...,0.400361,0.160266,0.310921,0.389470,0.267559,0.237281,0.377873,0.322401,0.869850,8.113634
1,2,B,B,D,F,A,O,0.743068,0.367411,1.021605,...,0.533087,0.558922,0.516294,0.594928,0.341439,0.906013,0.921701,0.261975,0.465083,8.481233
2,3,A,A,D,D,A,F,0.742708,0.310383,-0.012673,...,0.650609,0.375348,0.902567,0.555205,0.843531,0.748809,0.620126,0.541474,0.763846,8.364351
3,4,B,B,D,E,C,K,0.429551,0.620998,0.577942,...,0.668980,0.239061,0.732948,0.679618,0.574844,0.346010,0.714610,0.540150,0.280682,8.049253
4,6,A,A,D,E,A,N,1.058291,0.367492,-0.052389,...,0.686964,0.420667,0.648182,0.684501,0.956692,1.000773,0.776742,0.625849,0.250823,7.972260


In [6]:
# Separate target from features
y = train['target']
features = train.drop(['id','target'], axis=1)

# Preview features
features.head()

,cat0,cat1,cat5,cat7,cat8,cat9,cont0,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13
0,B,B,B,E,C,N,0.201470,-0.014822,0.669699,0.136278,0.610706,0.400361,0.160266,0.310921,0.389470,0.267559,0.237281,0.377873,0.322401,0.869850
1,B,B,D,F,A,O,0.743068,0.367411,1.021605,0.365798,0.276853,0.533087,0.558922,0.516294,0.594928,0.341439,0.906013,0.921701,0.261975,0.465083
2,A,A,D,D,A,F,0.742708,0.310383,-0.012673,0.576957,0.285074,0.650609,0.375348,0.902567,0.555205,0.843531,0.748809,0.620126,0.541474,0.763846
3,B,B,D,E,C,K,0.429551,0.620998,0.577942,0.280610,0.284667,0.668980,0.239061,0.732948,0.679618,0.574844,0.346010,0.714610,0.540150,0.280682
4,A,A,D,E,A,N,1.058291,0.367492,-0.052389,0.232407,0.287595,0.686964,0.420667,0.648182,0.684501,0.956692,1.000773,0.776742,0.625849,0.250823


# Preprocessing

In [7]:
'''
# List of categorical columns
object_cols = [col for col in features.columns if 'cat' in col]

# ordinal-encode categorical columns
X = features.copy()
X_test = test.drop(['id'], axis=1).copy()
ordinal_encoder = OrdinalEncoder()
X[object_cols] = ordinal_encoder.fit_transform(features[object_cols])
X_test[object_cols] = ordinal_encoder.transform(test[object_cols])

# Preview the ordinal-encoded features
X.head()
'''

"\n# List of categorical columns\nobject_cols = [col for col in features.columns if 'cat' in col]\n\n# ordinal-encode categorical columns\nX = features.copy()\nX_test = test.drop(['id'], axis=1).copy()\nordinal_encoder = OrdinalEncoder()\nX[object_cols] = ordinal_encoder.fit_transform(features[object_cols])\nX_test[object_cols] = ordinal_encoder.transform(test[object_cols])\n\n# Preview the ordinal-encoded features\nX.head()\n"

In [8]:

# List of categorical columns
object_cols = [col for col in features.columns if 'cat' in col]

# one-hot-encode categorical columns
X = features.copy()
X_test = test.drop(['id'], axis=1).copy()

oh_encoder = OneHotEncoder(sparse=False, handle_unknown="ignore")
X_ohe = oh_encoder.fit_transform(features[object_cols])
X_test_ohe = oh_encoder.transform(test[object_cols])

X_ohe = pd.DataFrame(X_ohe, columns=[f"ohe_{i}" for i in range(X_ohe.shape[1])])
X_test_ohe = pd.DataFrame(X_test_ohe, columns=[f"ohe_{i}" for i in range(X_test_ohe.shape[1])])

X = pd.concat([X, X_ohe], axis=1)
X_test = pd.concat([X_test, X_test_ohe], axis=1)
X = X.drop(object_cols, axis=1)
X_test = X_test.drop(object_cols, axis=1)
    
# Preview the one-hot-encoded features
X.head()


,cont0,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,...,ohe_28,ohe_29,ohe_30,ohe_31,ohe_32,ohe_33,ohe_34,ohe_35,ohe_36,ohe_37
0,0.201470,-0.014822,0.669699,0.136278,0.610706,0.400361,0.160266,0.310921,0.389470,0.267559,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.743068,0.367411,1.021605,0.365798,0.276853,0.533087,0.558922,0.516294,0.594928,0.341439,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.742708,0.310383,-0.012673,0.576957,0.285074,0.650609,0.375348,0.902567,0.555205,0.843531,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.429551,0.620998,0.577942,0.280610,0.284667,0.668980,0.239061,0.732948,0.679618,0.574844,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,1.058291,0.367492,-0.052389,0.232407,0.287595,0.686964,0.420667,0.648182,0.684501,0.956692,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [9]:
scale_features = [col for col in features.columns if 'cont' in col]

ss = StandardScaler()
X[scale_features] = ss.fit_transform(features[scale_features])
X_test[scale_features] = ss.transform(test[scale_features])

In [10]:
# Swap noise

# Random
def apply_noise_rn(df, p=.75):
    should_not_swap = np.random.binomial(1, p, df.shape)
    corrupted_df = df.where(should_not_swap == 1, np.random.permutation(df))
    return corrupted_df

# Row-wise
def apply_noise_row(df, p=.75):
    should_not_swap = np.zeros(df.shape)
    for i in range(df.shape[0]):
        for j in np.random.choice(df.shape[1],int(p*df.shape[1]),replace=False):
            should_not_swap[i,j]=1 
    corrupted_df = df.where(should_not_swap == 1, np.random.permutation(df))
    return corrupted_df

# Pseudo Label

In [11]:
pseudo = pd.read_csv("../input/30dml-pseudo2/xgb_submission_k.txt")[target]
train_pseudo = pd.concat([X, y], axis=1)
test_pseudo = pd.concat([X_test, pseudo], axis=1)
all_pseudo = pd.concat([train_pseudo, test_pseudo]).reset_index(drop=True)

# Optuna

In [12]:
# Optuna for parameter search
!pip install -q optuna

import optuna
import pickle

In [13]:
# for the fixed learning rate, use the opt n iterations and tune the tree hyperparameters
def objective(trial, X=all_pseudo.iloc[:,:-1], y=all_pseudo[target]):
  """
  """
  param_space = {
              #'device':'gpu',  # Use GPU acceleration
               #'boosting_type': 'gbdt',
               'reg_lambda':trial.suggest_uniform('reg_lambda', 72.656, 92.656),
              'reg_alpha':trial.suggest_uniform('reg_alpha', 37.17, 49.17),
                'subsample': trial.suggest_uniform('subsample', 0.016, 0.216),
              'colsample_bytree':trial.suggest_uniform('colsample_bytree', 0.001, 0.161),
                #'subsample_freq': trial.suggest_int('subsample_freq', 1, 10),
               'learning_rate':trial.suggest_uniform('learning_rate', 1.7e-2, 6.7e-2),
                 'min_child_samples':trial.suggest_int('min_child_samples', 64, 88),
              'num_leaves':trial.suggest_int('num_leaves', 2, 52),
              'max_depth':trial.suggest_int('max_depth', 21, 29),
              #'min_split_gain': trial.suggest_float('min_split_gain', 0.0, 0.005),
              #'class_weight':trial.suggest_categorical('class_weight',['balanced',None]),
               'n_estimators':N_ESTIMATORS,
                'n_jobs' : -1,
              'metric':'rmse',
              'max_bin':trial.suggest_int('max_bin', 1034, 1234),
              'cat_smooth':trial.suggest_int('cat_smooth', 50, 74),
              'cat_l2':trial.suggest_loguniform('cat_l2', 1e-4, 20)
              }
            
  seed_list=[SEED, SEED+1]
  #kf = KFold(n_splits=N_SPLITS, shuffle=True, random_state=CVSEED)
  lgb_oof = np.zeros(train.shape[0])
  #lgb_pred = np.zeros(test.shape[0])
  X_tr, X_va, y_tr, y_va = train_test_split(X,y,test_size=.2,random_state=CVSEED)
  trn_idx = X_tr.index
  val_idx = X_va.index
  
  #for (trn_idx, val_idx) in enumerate(kf.split(X, y)):
  #print(f"===== fold {fold} =====")
  oof_idx = np.array([idx for idx in val_idx if idx < train.shape[0]])
  preds_idx = np.array([idx for idx in val_idx if idx >= train.shape[0]])

  X_train, y_train = all_pseudo.iloc[trn_idx, :-1], all_pseudo[target].iloc[trn_idx]
  #X_train = apply_noise_rn(X_train)

  X_valid, y_valid = all_pseudo.iloc[oof_idx, :-1], all_pseudo[target].iloc[oof_idx]
  #X_test = all_pseudo[scale_features].iloc[preds_idx]
  
  #start = time.time()
  for inseed in seed_list:
    param_space['random_state'] = inseed

    model = lgb.LGBMRegressor(**param_space)
    model.fit(
        X_train, 
        y_train,
        eval_set=[(X_valid, y_valid)],
        eval_metric='rmse',
        early_stopping_rounds=EARLY_STOPPING_ROUNDS,
        verbose=VERBOSE,
        #categorical_feature=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
        #callbacks=[optuna.integration.LightGBMPruningCallback(trial, metric='rmse')],
    )


    lgb_oof[oof_idx] += model.predict(X_valid) / len(seed_list)
    #lgb_pred[preds_idx-train.shape[0]] += model.predict(X_test) / len(seed_list)

  #elapsed = time.time() - start
  rmse = mean_squared_error(y_valid, lgb_oof[oof_idx], squared=False)
  #print(f"fold {fold} - lgb rmse: {rmse:.6f}, elapsed time: {elapsed:.2f}sec\n")            
  
  #print(f"oof lgb_rmse = {mean_squared_error(train[target], lgb_oof, squared=False)}")

  
  return rmse

In [14]:
study = optuna.create_study(direction='minimize')
study.optimize(objective,n_trials= 30)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

[I 2021-08-30 08:21:18,129] A new study created in memory with name: no-name-998ef369-b7e4-4819-ad4c-bc79ca8fa6bb


Training until validation scores don't improve for 200 rounds
[1000]	valid_0's rmse: 0.71828
[2000]	valid_0's rmse: 0.712486
[3000]	valid_0's rmse: 0.710745
[4000]	valid_0's rmse: 0.710006
[5000]	valid_0's rmse: 0.709612
[6000]	valid_0's rmse: 0.709432
[7000]	valid_0's rmse: 0.709325
[8000]	valid_0's rmse: 0.709266
[9000]	valid_0's rmse: 0.709226
[10000]	valid_0's rmse: 0.709202
[11000]	valid_0's rmse: 0.709187
[12000]	valid_0's rmse: 0.709176
[13000]	valid_0's rmse: 0.709168
[14000]	valid_0's rmse: 0.709163
[15000]	valid_0's rmse: 0.709158
[16000]	valid_0's rmse: 0.709156
[17000]	valid_0's rmse: 0.709154
[18000]	valid_0's rmse: 0.709152
[19000]	valid_0's rmse: 0.709151
[20000]	valid_0's rmse: 0.709151
Did not meet early stopping. Best iteration is:
[19993]	valid_0's rmse: 0.709151
Training until validation scores don't improve for 200 rounds
[1000]	valid_0's rmse: 0.717833
[2000]	valid_0's rmse: 0.712314
[3000]	valid_0's rmse: 0.710595
[4000]	valid_0's rmse: 0.709871
[5000]	valid_0's 

[I 2021-08-30 08:27:09,016] Trial 0 finished with value: 0.7090585555497817 and parameters: {'reg_lambda': 88.49508493375501, 'reg_alpha': 47.72198593683443, 'subsample': 0.21176485725120908, 'colsample_bytree': 0.025623463389039897, 'learning_rate': 0.0410725103162716, 'min_child_samples': 77, 'num_leaves': 35, 'max_depth': 22, 'max_bin': 1150, 'cat_smooth': 54, 'cat_l2': 0.5077264020863262}. Best is trial 0 with value: 0.7090585555497817.


Training until validation scores don't improve for 200 rounds
[1000]	valid_0's rmse: 0.710651
[2000]	valid_0's rmse: 0.709415
[3000]	valid_0's rmse: 0.709122
[4000]	valid_0's rmse: 0.709063
Early stopping, best iteration is:
[4092]	valid_0's rmse: 0.709052
Training until validation scores don't improve for 200 rounds
[1000]	valid_0's rmse: 0.710718
[2000]	valid_0's rmse: 0.709426
[3000]	valid_0's rmse: 0.709125
[4000]	valid_0's rmse: 0.709034
[5000]	valid_0's rmse: 0.708984
Early stopping, best iteration is:
[5118]	valid_0's rmse: 0.708981


[I 2021-08-30 08:30:24,688] Trial 1 finished with value: 0.7089191505238409 and parameters: {'reg_lambda': 78.30774607437252, 'reg_alpha': 41.47943933361216, 'subsample': 0.03403231138207535, 'colsample_bytree': 0.13955134993591597, 'learning_rate': 0.032831791173961844, 'min_child_samples': 87, 'num_leaves': 28, 'max_depth': 25, 'max_bin': 1234, 'cat_smooth': 66, 'cat_l2': 0.18283125795704744}. Best is trial 1 with value: 0.7089191505238409.


...

Training until validation scores don't improve for 200 rounds
[1000]	valid_0's rmse: 0.716701
[2000]	valid_0's rmse: 0.71174
[3000]	valid_0's rmse: 0.710164
[4000]	valid_0's rmse: 0.709496
[5000]	valid_0's rmse: 0.70916
[6000]	valid_0's rmse: 0.70897
[7000]	valid_0's rmse: 0.708857
[8000]	valid_0's rmse: 0.708798
[9000]	valid_0's rmse: 0.708741
[10000]	valid_0's rmse: 0.708713
Early stopping, best iteration is:
[9968]	valid_0's rmse: 0.708712
Training until validation scores don't improve for 200 rounds
[1000]	valid_0's rmse: 0.716536
[2000]	valid_0's rmse: 0.711639
[3000]	valid_0's rmse: 0.710074
[4000]	valid_0's rmse: 0.709415
[5000]	valid_0's rmse: 0.709138
[6000]	valid_0's rmse: 0.708989
[7000]	valid_0's rmse: 0.708886
[8000]	valid_0's rmse: 0.708822
[9000]	valid_0's rmse: 0.708782
[10000]	valid_0's rmse: 0.708761
[11000]	valid_0's rmse: 0.708741
Early stopping, best iteration is:
[11019]	valid_0's rmse: 0.708739


[I 2021-08-30 10:44:26,032] Trial 28 finished with value: 0.7086560650150023 and parameters: {'reg_lambda': 74.59756823390178, 'reg_alpha': 44.76352524124459, 'subsample': 0.09038773853479755, 'colsample_bytree': 0.06636197658002779, 'learning_rate': 0.0176111653240539, 'min_child_samples': 67, 'num_leaves': 39, 'max_depth': 22, 'max_bin': 1192, 'cat_smooth': 65, 'cat_l2': 0.00010985676566472491}. Best is trial 14 with value: 0.7085806386132834.


Training until validation scores don't improve for 200 rounds
[1000]	valid_0's rmse: 0.725175
[2000]	valid_0's rmse: 0.717975
[3000]	valid_0's rmse: 0.714524
[4000]	valid_0's rmse: 0.712679
[5000]	valid_0's rmse: 0.711456
[6000]	valid_0's rmse: 0.710763
[7000]	valid_0's rmse: 0.710298
[8000]	valid_0's rmse: 0.709997
[9000]	valid_0's rmse: 0.709775
[10000]	valid_0's rmse: 0.709621
[11000]	valid_0's rmse: 0.709514
[12000]	valid_0's rmse: 0.709431
[13000]	valid_0's rmse: 0.709365
[14000]	valid_0's rmse: 0.709313
[15000]	valid_0's rmse: 0.709272
[16000]	valid_0's rmse: 0.709244
[17000]	valid_0's rmse: 0.709224
[18000]	valid_0's rmse: 0.709205
[19000]	valid_0's rmse: 0.70919
[20000]	valid_0's rmse: 0.70918
Did not meet early stopping. Best iteration is:
[19997]	valid_0's rmse: 0.70918
Training until validation scores don't improve for 200 rounds
[1000]	valid_0's rmse: 0.724705
[2000]	valid_0's rmse: 0.717678
[3000]	valid_0's rmse: 0.714213
[4000]	valid_0's rmse: 0.712317
[5000]	valid_0's rm

[I 2021-08-30 10:50:26,614] Trial 29 finished with value: 0.7091161429358452 and parameters: {'reg_lambda': 75.41519492696808, 'reg_alpha': 45.940896183660904, 'subsample': 0.13509468092939003, 'colsample_bytree': 0.027237869790237325, 'learning_rate': 0.021411313306400552, 'min_child_samples': 77, 'num_leaves': 26, 'max_depth': 24, 'max_bin': 1163, 'cat_smooth': 57, 'cat_l2': 0.8375836219734759}. Best is trial 14 with value: 0.7085806386132834.


Number of finished trials: 30
Best trial: {'reg_lambda': 73.91304305508143, 'reg_alpha': 39.457660766206786, 'subsample': 0.09889595934762334, 'colsample_bytree': 0.06241174534154789, 'learning_rate': 0.018268165672734094, 'min_child_samples': 68, 'num_leaves': 22, 'max_depth': 23, 'max_bin': 1200, 'cat_smooth': 69, 'cat_l2': 0.0005861671847540497}


In [15]:
study.best_params

{'reg_lambda': 73.91304305508143,
 'reg_alpha': 39.457660766206786,
 'subsample': 0.09889595934762334,
 'colsample_bytree': 0.06241174534154789,
 'learning_rate': 0.018268165672734094,
 'min_child_samples': 68,
 'num_leaves': 22,
 'max_depth': 23,
 'max_bin': 1200,
 'cat_smooth': 69,
 'cat_l2': 0.0005861671847540497}

# Log

====== Ordinal encoding =========

0.7123367407151787 no noise ver1 final

0.7158040228380602 row-wise noise ver2

0.7157995952863279 random noise ver3

====== One-hot encoding =========

0.7203460513257339 no noise ver4

0.7090027494545778 no noise ver9

0.7089277083045002 no noise ver10 (narrow space)

0.7088992777144029 no noise ver11 (narrow space)

0.7088362570431959 no noise ver12 (narrow space)

0.7087611371131594 no noise ver13 (narrow space) final

========================================== 
=========== Changed pseudo ===============

====== Ordinal encoding =========

0.7128954021404443 no noise ver17

====== One-hot encoding =========

0.708946431754134 no noise ver16

0.708661625542908 no noise ver18 (narrow space)
